<a href="https://colab.research.google.com/github/gacerioni/redis-workshop-notebook-validator/blob/master/redis-workshop-setup-notebook-validator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparando o ambiente para o Workshop

![Redis](https://redis.com/wp-content/themes/wpx/assets/images/logo-redis.svg?auto=webp&quality=85,75&width=120)

Este notebook, assim como os outros aqui no tutorial, foram desenhados para rodar com Redis Stack ou Redis Cloud. Isso simplifica as coisas, sem adicionar complexidade alguma.
Entretanto, podemos usar a instância de Redis que iremos instalar durante o SETUP (este notebook que você está lendo agora).

Para provisionar uma instância free-forever do Redis Cloud, e eu acredito que isso seja uma ótima ideia:
- Acesse https://redis.com/try-free/
- Registre com o seu email (gmail é uma boa pedida)
- Crie a subscription with do free tier (não precisa de cartão de crédito, sem gotchas)
- Crie um db habilitando as features do RedisStack (JSON, por exemplo).

In [ ]:
# Vamos instalar o redis localmente aqui no Colab
!pip install -q redis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 4.2 MB/s eta 0:00:00


### Instale a Redis Stack localmente (vai fazer sentido!)

Mesmo que você esteja usando o Redis Cloud, vamos configurar a cli aqui. A cli tool é conhecia simplesmente como `redis-cli`.

In [ ]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes


deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


### Configurando e testando a conexão com o seu Redis escolhido

Nenhuma alteração é necessária caso você esteja seguindo o tutorial usando o Redis que foi instalado aqui, localmente.
Caso você esteja utilizando Redis Cloud - coloque o endpoint host, port, e as credenciais pertinentes ao seu setup.

In [ ]:
import os

REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#Substitua os valores acima com os pertinentes, caso esteja no Redis Cloud. Será fácil do mesmo jeito, podem ficar tranquilos.
# Exemplo:
#REDIS_HOST="redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=18374
#REDIS_PASSWORD="1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# Caso o SSL esteja ativo pro endpoint, adicione --tls
# Recomendo não misturar lé com cré aqui, visto que não vamos ter nenhuma informação sensível passando pelo fio.
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# Caso o SSL esteja ativo pro endpoint, use rediss:// como o prefixo da URL
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"


### Checkpoint - Teste básico direto via Redis CLI

In [ ]:
!redis-cli $REDIS_CONN PING
!redis-cli $REDIS_CONN SET hello world
!redis-cli $REDIS_CONN GET hello

PONG
OK
"world"


### Checkpoint - Teste básico do Python client (que conecta com o Redis)

In [ ]:
import redis
r = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)

r.get("redis_da_massa")

### TTL - Time To Live - Just for fun

Um sistema de Caching robusto provavelmente estará gerenciando quanto tempo cada valor fica no cache. Temos libs, SDKs, e o RDI pra facilitar isso, mas vamos deixar esse papo pra outro workshop. Só lembrar que existem opções com o Redis (RDI) onde você nem precisaria cuidar do Cache Layer.

Enfim, vamos brincar aqui só pra garantir que o Setup está pronto pro Samba!

Redis permite que você controle esses comportamentos de TTL utilizando flags, como EX/EXAT/etc.

Para mais informações sobre este tema tão importante, veja (depois) https://redis.io/commands/set/

In [ ]:
import time
r.set(name="ttl_key", value="essa chave vai pro beleleu em breve...", ex=10)
for i in range(15):
  print(r.get(name="ttl_key"))
  time.sleep(1)

b'essa chave vai pro belel\xc3\xa9u em breve...'
b'essa chave vai pro belel\xc3\xa9u em breve...'
b'essa chave vai pro belel\xc3\xa9u em breve...'
b'essa chave vai pro belel\xc3\xa9u em breve...'
b'essa chave vai pro belel\xc3\xa9u em breve...'
b'essa chave vai pro belel\xc3\xa9u em breve...'
b'essa chave vai pro belel\xc3\xa9u em breve...'
b'essa chave vai pro belel\xc3\xa9u em breve...'
b'essa chave vai pro belel\xc3\xa9u em breve...'
b'essa chave vai pro belel\xc3\xa9u em breve...'
None
None
None
None
None
